<a href="https://colab.research.google.com/github/O-Kpy/Dacon/blob/main/Dacon_%EC%9C%84%EC%84%B1_%EC%98%81%EC%83%81%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EB%B6%81%EA%B7%B9_%ED%95%B4%EB%B9%99_%EC%98%88%EC%B8%A1_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C_%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import os 
from glob import glob
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torchvision import transforms, utils
from torchsummary import summary 
device='cuda' if torch.cuda.is_available() else 'cpu'
print('Using PyTorch version:',  torch.__version__, 'and Device:', device)

Using PyTorch version: 1.8.1+cu101 and Device: cuda


In [7]:
pip install catboost

     |████████████████████████████████| 69.2MB 43kB/s 


In [8]:
import catboost

model=catboost.CatBoostRegressor()
model.fit(eval_metrics)

NameError: ignored

In [ ]:
def missing_data(df):
    total = df.isnull().sum()
    percent = round(df.isnull().sum()/len(df)*100, 2)
    
    table = pd.concat([total, percent], axis=1, keys=['total', 'percent']).sort_index(by='total', ascending=False)
    return table

missing_data(train)

# 데이터 불러오기 주소 (os.path.join())

In [5]:
data_path = '/content/drive/MyDrive/dataset/Dacon/연습/제 2회 컴퓨터비전 학습 경진대회/'
train_data_path = os.path.join(data_path, 'train')
file_list = os.listdir(train_data_path)
file_list.sort()

In [18]:
data_path = '/content/drive/MyDrive/dataset/Dacon/연습/제 2회 컴퓨터비전 학습 경진대회/'
train_data_path = os.path.join(data_path, 'train')
print(train_data_path)
file_list = os.listdir(train_data_path)
file_list.sort()

/content/drive/MyDrive/dataset/Dacon/연습/제 2회 컴퓨터비전 학습 경진대회/train


In [24]:
class SealceDataset(Dataset):
  def __init__(self, data_dir, transform, data_type='train', frame_num=6, predict_num=2, stride=1):
    super(SealceDataset, self).__init__()

    data_to_path = os.path.join(data_dir, data_type)
    filenames = os.listdir(data_to_path)
    self.filepaths = [os.path.join(data_to_path, filename) for filename in sorted(filenames)]

    self.transform = transform
    self.frame_num = frame_num
    self.predict_num = predict_num
    self.stride = stride

  def __len__(self):
    return len(self.filepaths) - (self.frame_num + self.predict_num - 1) * self.stride
  
  def __getitem__(self, idx):
    dataset = []
    for id in range(idx, idx + self.frame_num + self.predict_num, self.stride):
      cur_npy = np.load(self.filepaths[id])[:,:,0]/250    # 250을 나눠주어 저장하지 않으면 toTensor했을때 오차값이 크게 생겼습니다
      cur_tensor = self.transform(cur_npy)                # tensor로 저장
      dataset.append(cur_tensor)
    x = torch.stack(dataset[:self.frame_num])
    x = x.transpose(0,1).to(dtype=torch.float)              # [1, 6, 448, 304] => [channel, frames, height, width]
    y = torch.stack(dataset[self.frame_num:])               
    y = y.transpose(0,1)                                    # [1, 2, 448, 304] => [channel, frames, height, width]
    return x, y

def get_transform():
  return transforms.Compose([
                             transforms.ToTensor()
  ])

transform = get_transform()

ice_dataset = SealceDataset(data_path, transform, 'train', 6, 2, 1)

In [26]:
a,b = ice_dataset[1]
a.shape, b.shape

(torch.Size([1, 6, 448, 304]), torch.Size([1, 2, 448, 304]))

In [21]:
torch.set_printoptions(threshold=10000) # show all tensor without abbreviation

class SeaIceDataset(Dataset):
    def __init__(self, data_dir, transform, data_type="train", frame_num=6, predict_num=2, stride=1):
        super(SeaIceDataset, self).__init__()

        """
        data_dir                => data folder path
        transform               => data to tensor
        data_type="train"       => choose train / valid / test
        frame_num               => frame nums to use on train 
        predict_num             => frame nums to predict
        stride_num              => stride for frames (if stride=2 => 197811.npy, 198001.npy, 198003.npy ... )
                                   만약 8월끼리 비교하고 싶다면 stride = 12 를 넣어준다.
        """

        data_to_path = os.path.join(data_dir, data_type)
        filenames = os.listdir(data_to_path)
        self.filepaths = [os.path.join(data_to_path, filename) for filename in sorted(filenames)]
        
        self.transform = transform  # numpy 배열을 tensor 배열로 바꿔주는 함수
        self.frame_num = frame_num 
        self.predict_num = predict_num
        self.stride = stride

    def __len__(self):
        # len = dataset으로 시작가능한 인덱스 번호 
        return len(self.filepaths) - (self.frame_num + self.predict_num - 1) * self.stride
    
    def __getitem__(self, idx):
        """
        it will return (x_with_frame_num, y_true_with_predict_num)
        if frame_num = 6, predict_num = 2
        ((6, 1, 448, 304), (2, 1, 448, 304))
        """
        dataset = []
        for id in range(idx, idx + self.frame_num + self.predict_num, self.stride):
            cur_npy = np.load(self.filepaths[id])[:,:,0]/250    # 250을 나눠주어 저장하지 않으면 toTensor했을때 오차값이 크게 생겼습니다
            cur_tensor = self.transform(cur_npy)                # tensor로 저장
            dataset.append(cur_tensor)
        x = torch.stack(dataset[:self.frame_num])
        x = x.transpose(0,1).to(dtype=torch.float)              # [1, 6, 448, 304] => [channel, frames, height, width]
        y = torch.stack(dataset[self.frame_num:])               
        y = y.transpose(0,1)                                    # [1, 2, 448, 304] => [channel, frames, height, width]
        return x, y

def getTransform():
    return transforms.Compose([transforms.ToTensor()])



transform = getTransform()

ice_dataset = SeaIceDataset(data_path, transform, "train", 6, 2, 1)

a,b = ice_dataset[1]        # sample to see 
print(len(ice_dataset))     # 데이터셋에 있는 총 데이터의 개수는 8개씩 묶여있는 475개의 데이터가 있습니다
print(a.shape, b.shape)

475
torch.Size([1, 6, 448, 304]) torch.Size([1, 2, 448, 304])


In [27]:
len_ice_dataset=len(ice_dataset)
len_ice_train = int(0.8*len_ice_dataset)
len_ice_valid = len_ice_dataset - len_ice_train

train_dataset, valid_dataset = random_split(ice_dataset, [len_ice_train, len_ice_valid])  # random_split을 한다. ice_dataset을 train개수와 valid개수 만큼

for x, y in train_dataset:
  print(x.shape, y.shape)
  break

# 잘 나뉘어졌다

torch.Size([1, 6, 448, 304]) torch.Size([1, 2, 448, 304])


In [ ]:
# Create splited Dataset to Train and Valid

len_ice_dataset = len(ice_dataset)
len_ice_train = int(0.8*len_ice_dataset)
len_ice_valid = len_ice_dataset - len_ice_train


train_dataset, valid_dataset = random_split(ice_dataset, [len_ice_train, len_ice_valid])
print(f"train dataset length : {len(train_dataset)}")
print(f"valid dataset length : {len(valid_dataset)}")

# show one of train_ds
for x, y in train_dataset:
    print(x.shape, y.shape)
    break

train dataset length : 380
valid dataset length : 95
torch.Size([1, 6, 448, 304]) torch.Size([1, 2, 448, 304])


In [29]:
batch_size = 12

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

for x, y in train_dataloader:
  print(x.shape, y.shape)
  break

torch.Size([12, 1, 6, 448, 304]) torch.Size([12, 1, 2, 448, 304])


In [ ]:
# Creating DataLoader
# too slow now...

BATCH_SIZE = 12

# Dataloader 클래스는 데이터셋에서 배치 개수만큼 뽑아서 제공해줍니다
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

# show example
# train_dataloader length => 32
for x, y in train_dataloader:
    print(x.shape, y.shape)
    break

torch.Size([12, 1, 6, 448, 304]) torch.Size([12, 1, 2, 448, 304])


# Building Model 모델 구성

In [31]:
model_params = {'shape':(6,1,448,304),
                'init_filters':8,
                'dropout_rate':0.5}

# Creating Model

class CustomNet(nn.Module):
    def __init__(self, params):
        super(CustomNet, self).__init__()
        input_frames, input_channel, input_height, input_width = params["shape"]
        init_filters = params["init_filters"]
        self.dropout_rate = params["dropout_rate"]
        self.conv1 = nn.Conv3d(input_channel, init_filters, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(init_filters, init_filters*2, kernel_size=3, padding=1)
        self.conv3 = nn.ConvTranspose3d(init_filters*2, 1, kernel_size=3, padding=1)

    def forward(self, x):
        input = x
        x = F.relu(self.conv1(x))
        x = F.max_pool3d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.upsample(x, size=(2, 448, 304))
        print("input: ", input.shape)
        print("output: ", x.shape)
        return x

  # Linear가 없네?

In [35]:
my_model = CustomNet(model_params).to(device)
print(my_model)
summary(my_model, input_size=(1,6,448,304), device=device)

CustomNet(
  (conv1): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv2): Conv3d(8, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3): ConvTranspose3d(16, 1, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
)
input:  torch.Size([2, 1, 6, 448, 304])
output:  torch.Size([2, 1, 2, 448, 304])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 8, 6, 448, 304]             224
            Conv3d-2      [-1, 16, 3, 224, 152]           3,472
   ConvTranspose3d-3       [-1, 1, 3, 224, 152]             433
Total params: 4,129
Trainable params: 4,129
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 3.12
Forward/backward pass size (MB): 63.12
Params size (MB): 0.02
Estimated Total Size (MB): 66.26
----------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [38]:
# Loss Function && metric Function

# metrics
def mae_score(true, pred):
    true, pred = numpy_to_tensor(true, pred)
    score = np.mean(np.abs(true-pred))
    
    return score

# metrics
def f1_score(true, pred):
    true, pred = numpy_to_tensor(true, pred)

    target = np.where((true>1*0.05)<1*0.5)
    
    true = true[target]
    pred = pred[target]
    true = np.where(true < 1*0.15, 0, 1)
    pred = np.where(pred < 1*0.15, 0, 1)
    
    right = np.sum(true * pred == 1)
    precision = right / np.sum(true+1e-8)
    recall = right / np.sum(pred+1e-8)
    score = 2 * precision*recall/(precision+recall+1e-8)
    
    return score
    
# loss function
def mae_over_f1(true, pred):
    mae = mae_score(true, pred)
    f1 = f1_score(true, pred)
    score = mae/(f1+1e-8)
    
    return score

def numpy_to_tensor(true, pred):
    return true.cpu().detach().numpy(), pred.cpu().detach().numpy()

In [37]:
# Optimizer

# adam optimizer
opt_adam = optim.Adam(my_model.parameters(), lr=3e-4)

def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group["lr"]

# check our learning rate
current_lr = get_lr(opt_adam)
print(f"current_lr = {current_lr}")



# learning rate scheduler
lr_scheduler = ReduceLROnPlateau(opt_adam, mode="min", factor=0.5, patience=20, verbose=1)

# example 
for i in range(100):
    lr_scheduler.step(i)

current_lr = 0.0003
Epoch    22: reducing learning rate of group 0 to 1.5000e-04.
Epoch    43: reducing learning rate of group 0 to 7.5000e-05.
Epoch    64: reducing learning rate of group 0 to 3.7500e-05.
Epoch    85: reducing learning rate of group 0 to 1.8750e-05.


# Training Setting
에폭 한번 할때마다 loss_epoch함수

dataloader의 x,y 마다 metric_batch를 통해 metric값을 계산

loss_batch를 통해 loss값을 계산

In [44]:
def metrics_batch(pred, true, metrics):
  if metrics:
    return list(map(lambda x: x(true, pred), metrics))
    mae_score = mae_score(true, pred)
    f1_score = f1_score(true, pred)
    return (mae_score, f1_score)

def loss_batch(loss_func, pred, true, opt=None):
  loss = loss_func(true, pred)
  with torch.no_grad():
    metrics = metrics_batch(pred, true, [mae_score, f1_score])
  if opt is not None:
    opt.zero_grad()
    #loss.backward()
    opt.step()
  return loss, metrics

def loss_epoch(model, loss_func, dataset_dataloader, sanity_check=False, opt=None):
  running_loss = 0.0
  running_metric = [0.0, 0.0]
  len_data = len(dataset_dataloader.dataset)

  for x,y in dataset_dataloader:
    x = x.to(device)
    y = y.to(device)

    pred = model(x)
    loss, metrics = loss_batch(loss_func, pred, y, opt)
    running_loss += loss
    if metrics is not None:
      for idx, metric_value in enumerate(metrics):
        running_metric[idx] += metric_value
        
        # 문제 있으면 break, 여기서는 True 일때 바로 break
    if sanity_check is True:
      break
    
  loss = running_loss / float(len_data)
  metrics = list(map(lambda x: x/float(len_data), metrics))
  print(loss, metrics)
  return loss, metrics

In [ ]:
# Training 

def metrics_batch(pred, true, metrics):
    # if needed add param "metrics" to custom
    """
    output will be pred
    target will be corrects
    """
    if metrics:
        return list(map(lambda x: x(true, pred), metrics))
    mae_score = mae_score(true, pred)
    f1_score = f1_score(true, pred)
    return (mae_score, f1_score)

def loss_batch(loss_func, pred, true, opt=None):
    """
    loss_func => mae_over_f1
    """
    loss = loss_func(true, pred)
    with torch.no_grad():
        metrics = metrics_batch(pred, true, [mae_score, f1_score])
    if opt is not None:
        opt.zero_grad()
        # loss.backward()
        opt.step()  # 학습이 이뤄지는 곳
    return loss, metrics

def loss_epoch(model, loss_func, dataset_dataloader, sanity_check=False, opt=None):
    running_loss = 0.0
    running_metric = [0.0, 0.0]
    len_data = len(dataset_dataloader.dataset)

    for x, y in dataset_dataloader:
        x = x.to(device)
        y = y.to(device)
        # 모델 결과
        pred = model(x)
        # 손실함수 구하기
        loss, metrics = loss_batch(loss_func, pred, y, opt)
        # 손실함수 
        running_loss += loss
        if metrics is not None:
            for idx, metric_value in enumerate(metrics):
                running_metric[idx] += metric_value
        
        # 문제 있으면 break, 여기서는 True 일때 바로 break
        if sanity_check is True:
            break
    
    loss = running_loss / float(len_data)
    metrics = list(map(lambda x: x/float(len_data), metrics))
    print(loss, metrics)
    return loss, metrics

In [40]:
loss_func = mae_over_f1
opt_adam = optim.Adam(my_model.parameters(), lr=3e-4)
lr_scheduler = ReduceLROnPlateau(opt_adam, mode="min", factor=0.5, patience=20, verbose=1)

TRAIN_PARAMS = {
    "num_epochs" : 10,
    "loss_func" : loss_func,
    "optimizer" : opt_adam,
    "train_dataloader" : train_dataloader,
    "valid_dataloader" : valid_dataloader,
    "sanity_check" : True,
    "lr_scheduler" : lr_scheduler,
    "save_path" : "./weights.pt"
}

In [45]:
def train(model, params):
    num_epochs = params['num_epochs']
    loss_func = params['loss_func']
    opt = params["optimizer"]
    train_dataloader = params['train_dataloader']
    valid_dataloader = params['valid_dataloader']
    sanity_check = params['sanity_check']
    lr_scheduler = params['lr_scheduler']
    save_path = params['save_path']

    # keep history of the loss and metric
    loss_hist = {
        "train" : [],
        "valid" : []
    }

    metrics_hist = {
        "train" : [],
        "valid" : []
    }

    # copy best weights
    best_model_weights = copy.deepcopy(model.state_dict())
    # init best loss
    best_loss = float("inf")

    for epoch in range(num_epochs):
        current_lr = get_lr(opt)
        print(f'Epoch:{epoch}/{num_epochs-1}, current lr:{current_lr}')
        model.train()
        train_loss, train_metrics = loss_epoch(model, loss_func, train_dataloader, sanity_check, opt)

        # save history
        loss_hist["train"].append(train_loss)
        metrics_hist["train"].append(train_metrics)

        # model.eval()
        # with torch.no_grad():
    

    return model, loss_hist, metrics_hist


my_model, loss_hist, metrics_hist = train(my_model, TRAIN_PARAMS)

print(loss_hist)

Epoch:0/9, current lr:0.0003


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


input:  torch.Size([12, 1, 6, 448, 304])
output:  torch.Size([12, 1, 2, 448, 304])
34149.98191103034 [0.00034149981911030345, 0.0]
Epoch:1/9, current lr:0.0003
input:  torch.Size([12, 1, 6, 448, 304])
output:  torch.Size([12, 1, 2, 448, 304])
34149.98191176965 [0.00034149981911769657, 0.0]
Epoch:2/9, current lr:0.0003
input:  torch.Size([12, 1, 6, 448, 304])
output:  torch.Size([12, 1, 2, 448, 304])
34149.98191147273 [0.0003414998191147273, 0.0]
Epoch:3/9, current lr:0.0003
input:  torch.Size([12, 1, 6, 448, 304])
output:  torch.Size([12, 1, 2, 448, 304])
34149.98191253537 [0.00034149981912535364, 0.0]
Epoch:4/9, current lr:0.0003
input:  torch.Size([12, 1, 6, 448, 304])
output:  torch.Size([12, 1, 2, 448, 304])
34149.98191152611 [0.0003414998191152611, 0.0]
Epoch:5/9, current lr:0.0003
input:  torch.Size([12, 1, 6, 448, 304])
output:  torch.Size([12, 1, 2, 448, 304])
34149.981909977 [0.00034149981909977004, 0.0]
Epoch:6/9, current lr:0.0003
input:  torch.Size([12, 1, 6, 448, 304])
out